# Cleaning the Sentiment140 data


The [Sentiment140](http://www.sentiment140.com/) dataset is a collection of 1.6 million tweets that have been tagged as either positive or negative.

Before we clean it, a question: _how'd they get so many tagged tweets?_ If you poke around on their documentation, [the answer is hiding right here](http://help.sentiment140.com/for-students):

> In our approach, we assume that any tweet with positive emoticons, like :), were positive, and tweets with negative emoticons, like :(, were negative.

That's a good thing to discuss later, but for now let's just clean it up. In this notebook we'll be removing columns we don't want, and standardizing the sentiment column.

<p class="reading-options">
  <a class="btn" href="/investigating-sentiment-analysis/cleaning-the-sentiment140-data">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/investigating-sentiment-analysis/notebooks/Cleaning the Sentiment140 data.ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/investigating-sentiment-analysis/notebooks/Cleaning the Sentiment140 data.ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

### Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.
* **training.1600000.processed.noemoticon.csv:** raw data from Sentiment140 - 1.4 million tweets tagged for sentiment, no column headers, nothing cleaned up


In [1]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip

--2022-12-07 15:19:20--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  34.2MB/s    in 2.4s    

2022-12-07 15:19:22 (34.2 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  


## Read the tweets in

In [3]:
import pandas as pd

df = pd.read_csv("data/training.1600000.processed.noemoticon.csv",
                names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                encoding='latin-1')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   polarity  1600000 non-null  int64 
 1   id        1600000 non-null  int64 
 2   date      1600000 non-null  object
 3   query     1600000 non-null  object
 4   user      1600000 non-null  object
 5   text      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


## Update polarity

Right now the `polarity` column is `0` for negative, `4` for positive. Let's change that to `0` and `1` to make things a little more reasonably readable.

In [4]:
df.polarity.value_counts()

0    800000
4    800000
Name: polarity, dtype: int64

In [5]:
df.polarity = df.polarity.replace({0: 0, 4: 1})
df.polarity.value_counts()

0    800000
1    800000
Name: polarity, dtype: int64

## Remove unneeded columns

We don't need all those columns! Let's get rid of the ones that won't affect the sentiment.

In [6]:
df = df.drop(columns=['id', 'date', 'query', 'user'])
df.head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Sample

To make the filesize a little smaller and pandas a little happier, let's knock this down to 500,000 tweets.

In [7]:
df = df.sample(n=500000)
df.polarity.value_counts()

0    250152
1    249848
Name: polarity, dtype: int64

In [8]:
df.to_csv("data/sentiment140-subset.csv", index=False)

## Review

In this section, we cleaned up the **Sentiment140** tweet dataset. Sentiment140 is a collection of 1.6 million tweets that are marked as either positive or negative sentiment.